Python interface to PostgreSQL  # Rocky Wang

In [1]:
#!pip install -U "psycopg[binary]"

#### Connect to the local PostgreSQL database running as Docker container

In [1]:
import psycopg, os

print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="School",
    user="postgres",
    password="-----")

Connecting to the PostgreSQL database...


In [2]:
# create a cursor
cur = conn.cursor()

In [3]:
       
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)
       
createCmd = """ CREATE TABLE company (
                ID SERIAL PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                headquarter_city VARCHAR(255) NOT NULL,
                CEO VARCHAR(255) NOT NULL,
                founding_year INTEGER NOT NULL
                );
                
                CREATE TABLE CEO (
                ID SERIAL PRIMARY KEY,
                first_name VARCHAR(255) NOT NULL,
                last_name VARCHAR(255) NOT NULL,
                year_of_birth INTEGER NOT NULL,
                place_of_birth VARCHAR(255) NOT NULL,
                year_become_CEO INTEGER NOT NULL
                );
                
                CREATE TABLE city (
                city VARCHAR(255) PRIMARY KEY,
                state VARCHAR(255),
                country VARCHAR(255) NOT NULL
                )
            """
    
cur.execute(createCmd)
conn.commit()


PostgreSQL database version:
('PostgreSQL 14.1, compiled by Visual C++ build 1914, 64-bit',)


In [4]:
#insert records into company table

cur.execute("INSERT into company VALUES(1, 'Apple', 'Cupertino', 'Tim Cook', '1976')")
cur.execute("INSERT into company VALUES(2, 'IBM', 'Armonk', 'Arvind Krishna', '1922')")
cur.execute("INSERT into company VALUES(3, 'Amazon', 'Seattle', 'Andy Jassy', '1994')")
cur.execute("INSERT into company VALUES(4, 'General Motors', 'Detroit', 'Mary Barra', '1908')")
cur.execute("INSERT into company VALUES(5, 'Sony Corporation', 'Tokyo', 'Kenichiro Yoshida', '1946')")
cur.execute("INSERT into company VALUES(6, 'Alibaba', 'Hangzhou', 'Daniel Zhang', '1999')")
cur.execute("INSERT into company VALUES(7, 'Kering SA', 'Paris', 'François-Henri Pinault', '1963')")
cur.execute("INSERT into company VALUES(8, 'Samsung Electronics', 'Suwon', 'Lee Jae-yong', '1969')")
cur.execute("INSERT into company VALUES(9, 'Walmart ', 'Bentonville', 'Doug McMillon', '1962')")
conn.commit()

In [5]:
#insert records into CEO table

cur.execute("INSERT into CEO VALUES(1, 'Tim ', 'Cook', '1960', 'Mobile', '2011')")
cur.execute("INSERT into CEO VALUES(2, 'Arvind', 'Krishna', '1962', 'West Godavari', '2020')")
cur.execute("INSERT into CEO VALUES(3, 'Andy', 'Jassy', '1968', 'Scarsdale', '2021')")
cur.execute("INSERT into CEO VALUES(4, 'Mary', 'Barra', '1961', 'Royal Oak', '2014')")
cur.execute("INSERT into CEO VALUES(5, 'Kenichiro', 'Yoshida', '1959', 'Kumamoto', '2018')")
cur.execute("INSERT into CEO VALUES(6, 'Daniel', 'Zhang', '1972', 'Shanghai', '2015')")
cur.execute("INSERT into CEO VALUES(7, 'François-Henri', 'Pinault', '1962', 'Rennes', '2005')")
cur.execute("INSERT into CEO VALUES(8, 'Lee', 'Jae-yong', '1968', 'Seoul', '2022')")
cur.execute("INSERT into CEO VALUES(9, 'Doug', 'McMillon', '1966', 'Memphis', '2014')")
conn.commit()

In [6]:
#insert records into city table

cur.execute("INSERT into city VALUES('Mobile', 'AL', 'USA')")
cur.execute("INSERT into city VALUES('Cupertino', 'CA', 'USA')")
cur.execute("INSERT into city VALUES('West Godavari', 'Andhra-Pradesh', 'India')")
cur.execute("INSERT into city VALUES('Armonk', 'NY', 'USA')")
cur.execute("INSERT into city VALUES('Scarsdale', 'NY', 'USA')")
cur.execute("INSERT into city VALUES('Seattle', 'WA', 'USA')")
cur.execute("INSERT into city VALUES('Royal Oak', 'MI', 'USA')")
cur.execute("INSERT into city VALUES('Detroit', 'MI', 'USA')")
cur.execute("INSERT into city VALUES('Kumamoto', '', 'Japan')")

cur.execute("INSERT into city VALUES('Tokyo', '', 'Japan')")
cur.execute("INSERT into city VALUES('Shanghai', '', 'China')")
cur.execute("INSERT into city VALUES('Hangzhou', 'Zhejiang', 'China')")
cur.execute("INSERT into city VALUES('Rennes', 'Illes-et-Villaine', 'France')")
cur.execute("INSERT into city VALUES('Paris', '', 'France')")
cur.execute("INSERT into city VALUES('Seoul', '', 'South Korea')")
cur.execute("INSERT into city VALUES('Suwon', 'Gyeonggi-do', 'South Korea')")
cur.execute("INSERT into city VALUES('Memphis', 'TN', 'USA')")
cur.execute("INSERT into city VALUES('Bentonville', 'Arkansas', 'USA')")
conn.commit()

In [7]:
#What’s the age of the company whose CEO was born in the state of Tennessee?

queryCmd = """
            WITH CTE as (
            SELECT CONCAT(first_name,' ',last_name) AS ceo, state
            FROM CEO, city WHERE CEO.place_of_birth = city.city)

            SELECT (EXTRACT('Year' FROM CURRENT_DATE) - founding_year) AS age, name
            from company
            WHERE CEO = (SELECT ceo from CTE WHERE state='TN')
            """    
cur.execute(queryCmd)    


row = cur.fetchone()
while row is not None:
    print("Age:", row[0], " Company:", row[1])
    row = cur.fetchone()    

Age: 61  Company: Walmart 


In [9]:
#What are the full names of CEOs of companies founded in the USA before 1965?

queryCmd = """
            SELECT c.CEO, c.name, c.founding_year
            FROM company c JOIN city t ON c.headquarter_city = t.city
            WHERE t.country = 'USA' AND c.founding_year < 1965
            """    
cur.execute(queryCmd)    


row = cur.fetchone()
while row is not None:
    print("CEO:", row[0], " Company:", row[1], " Founding year:", row[2])
    row = cur.fetchone()       

CEO: Arvind Krishna  Company: IBM  Founding year: 1922
CEO: Mary Barra  Company: General Motors  Founding year: 1908
CEO: Doug McMillon  Company: Walmart   Founding year: 1962


In [10]:
#Which companies have CEOs who are younger than 60?

queryCmd = """
            WITH CTE as (
            SELECT CONCAT(first_name,' ',last_name) AS ceo, (EXTRACT('Year' FROM CURRENT_DATE) - year_of_birth) AS age
            FROM CEO )

            SELECT c.name, c.CEO, o.age
            FROM company c JOIN CTE o on c.CEO = o.ceo
            WHERE age < 60
            """    
cur.execute(queryCmd)    


row = cur.fetchone()
while row is not None:
    print("company:", row[0], " CEO:", row[1], " Age:", row[2])
    row = cur.fetchone()     

company: Amazon  CEO: Andy Jassy  Age: 55
company: Alibaba  CEO: Daniel Zhang  Age: 51
company: Samsung Electronics  CEO: Lee Jae-yong  Age: 55
company: Walmart   CEO: Doug McMillon  Age: 57


In [11]:
#What is the headquarter city of the company whose CEO was born in India?

queryCmd = """
            WITH CTE as (
            SELECT CONCAT(first_name,' ',last_name) AS ceo, state, country
            FROM CEO, city WHERE CEO.place_of_birth = city.city)

            SELECT c.headquarter_city, c.name, c.CEO, o.country
            FROM company c JOIN CTE o ON c.CEO = o.ceo
            WHERE o.country = 'India'
            """    
cur.execute(queryCmd)    


row = cur.fetchone()
while row is not None:
    print("Headquarter:", row[0], " Company:", row[1], " CEO:", row[2], " Borth conutry:", row[3])
    row = cur.fetchone()

Headquarter: Armonk  Company: IBM  CEO: Arvind Krishna  Borth conutry: India


In [12]:
#Which companies have CEOs who started in their positions after 2014? 

queryCmd = """
            SELECT c.name, c.CEO, o.year_become_ceo
            FROM company c, CEO o
            WHERE c.id = o.id AND o.year_become_ceo > 2014
            """    
cur.execute(queryCmd)    


row = cur.fetchone()
while row is not None:
    print("company:", row[0], " CEO:", row[1], " year_become_ceo:", row[2])
    row = cur.fetchone()   

company: IBM  CEO: Arvind Krishna  year_become_ceo: 2020
company: Amazon  CEO: Andy Jassy  year_become_ceo: 2021
company: Sony Corporation  CEO: Kenichiro Yoshida  year_become_ceo: 2018
company: Alibaba  CEO: Daniel Zhang  year_become_ceo: 2015
company: Samsung Electronics  CEO: Lee Jae-yong  year_become_ceo: 2022


#### Delete the table that you created to avoid errors trying to recreate the table

In [13]:
cur.execute("DROP TABLE company")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=School) at 0x1c6e6ef80f0>

In [14]:
cur.execute("DROP TABLE CEO")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=School) at 0x1c6e6ef80f0>

In [15]:
cur.execute("DROP TABLE city")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=School) at 0x1c6e6ef80f0>

In [16]:
# close the communication with the PostgreSQL
cur.close()